In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('/content/sample_data/mission 4.csv')

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
encode_cols = ['Brand', 'Processor Type', 'Operating System']
le = {}
for col in encode_cols:
  le[col] = LabelEncoder()
  data[col] = le[col].fit_transform(data[col])


In [ ]:
data.head()

In [ ]:
scale_Cols = ['RAM Size (GB)', 'Storage (GB)', 'Screen Size (inches)']
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[scale_Cols] = scaler.fit_transform(data[scale_Cols])

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
plt.figure(figsize=(10, 8))
sns.histplot(data['Price ($)'], kde=True, color="red")
plt.xlabel('Price ($)')
plt.ylabel('Frequency')
plt.title('Distribution of Laptop Prices')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(x=data['RAM Size (GB)'], y=data['Price ($)'], color='Blue')
plt.xlabel('Ram')
plt.ylabel('Price')
plt.title('RAM and Price Relationship')


In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x=data['Brand'], y=data['Price ($)'], color='Green')
plt.xlabel('Brand')
plt.ylabel('Price')
plt.title('Brand Impact on Price ')

In [ ]:
from sklearn.model_selection import train_test_split
x = data.drop('Price ($)', axis=1)
y = data['Price ($)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
dt = DecisionTreeRegressor()
dt.fit(x_train, y_train)
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
y_pred_lr = lr.predict(x_test)
y_pred_dt = dt.predict(x_test)
y_pred_rf = rf.predict(x_test)


In [ ]:
print("Linear Regression:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_lr))
print("R-squared:", r2_score(y_test, y_pred_lr))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred_lr))

print("Decision Tree Regression:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_dt))
print("R-squared:", r2_score(y_test, y_pred_dt))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred_dt))

print("Random Forest Regression:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_rf))
print("R-squared:", r2_score(y_test, y_pred_rf))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred_rf))

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
random_params = {
    'n_estimators': [50, 100, 200, 250,300],
    'max_depth': [None, 10, 20, 30,35,40],

}
random_model = RandomizedSearchCV(RandomForestRegressor(), random_params, cv=5)
random_model.fit(x_train, y_train)


In [ ]:
print("Best Parameters:", random_model.best_params_)
print("Best Score:", random_model.best_score_)

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
def predict_laptop_price(brand, processor_type, ram_size, storage, screen, os):
  try:
    input_data = pd.DataFrame(
        {
            'Brand': [brand],
            'Processor Type': [processor_type],
            'RAM Size (GB)': [ram_size],
            'Storage (GB)': [storage],
            'Screen Size (inches)': [screen],
            'Operating System': [os]

        }
    )

    for col in encode_cols:
      input_data[col] = le[col].transform(input_data[col])
    input_data[scale_Cols] = scaler.transform(input_data[scale_Cols])
    prediction = random_model.best_estimator_.predict(input_data)
    return prediction[0]
  except Exception as e:
    return str(e)
gr.Interface(
      inputs=[
          gr.Dropdown(choices=list(data['Brand'].unique()), label='Brand'),
          gr.Dropdown(choices=list(data['Processor Type'].unique()), label='Processor Type'),
          gr.Number(label='RAM Size (GB)'),
          gr.Number(label='Storage (GB)'),
          gr.Number(label='Screen Size (inches)'),
          gr.Dropdown(choices=list(data['Operating System'].unique()), label='Operating System')
      ]
      , outputs=gr.Textbox(label='Predicted Price ($)'),
      fn=predict_laptop_price,
      title='Laptop Price Prediction',
      description='Enter the details of the laptop to predict its price.'
  ).launch()

In [ ]:
data['Brand'].unique()

array(['Acer', 'Apple', 'HP', 'Lenovo', 'Dell'], dtype=object)